In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, f1_score
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


In [2]:
def compute_performance_metrics(y_test, y_pred, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1_score_value = f1_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} Recall: {recall}")
    print(f"{model_name} F1 Score: {f1_score_value}")

def compute_classification_metric(dataset, predictions, label_name_v, favorable_label_v, unfavorable_label_v, privileged_attribute, unprivileged_attributes):
    features = [privileged_attribute] + unprivileged_attributes

    aif_race_dataset = BinaryLabelDataset(
            df=dataset,
            favorable_label=favorable_label_v,
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute],
        )

    aif_race_pred = BinaryLabelDataset(
            df=predictions,
            favorable_label=favorable_label_v,
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute],
        )

    race_privileged_group = [{privileged_attribute: 1}]
    race_unprivileged_groups = [{attr: 1} for attr in unprivileged_attributes]

    fairness_metrics = ClassificationMetric(dataset=aif_race_dataset,
                            classified_dataset=aif_race_pred,
                            unprivileged_groups=race_unprivileged_groups,
                            privileged_groups=race_privileged_group)
    
    return fairness_metrics

def compute_fairness_metrics(fairness_metrics: ClassificationMetric):
    # Values less than 0 indicate that privileged group has higher
    # proportion of predicted positive outcomes than unprivileged group.
    # Value higher than 0 indicates that unprivileged group has higher proportion
    # of predicted positive outcomes than privileged group.
    SPD = round(fairness_metrics.statistical_parity_difference(),3)

    # Measures the deviation from the equality of opportunity, which means that the same
    # proportion of each population receives the favorable outcome. This measure must be equal to 0 to be fair.
    EOD = round(fairness_metrics.equal_opportunity_difference(),3)

    # Average of difference in False Positive Rate and True Positive Rate for unprivileged and privileged groups
    # A value of 0 indicates equality of odds, which means that samples in both the privileged and unprivileged
    # groups have the same probability of being classified positively.
    AOD = round(fairness_metrics.average_odds_difference(),3)

    print(f"Statistical Parity Difference (SPD): {SPD}")
    print(f"Average Odds Difference (AOD): {AOD}")
    print(f"Equal Opportunity Difference (EOD): {EOD}")

In [3]:
df_tidy = pd.read_csv("adult/adultTidy.csv")
df_tidy.head()
print("colonne ed entries",df_tidy.shape)

colonne ed entries (48843, 15)


In [4]:
# Drop rows with missing values because all the column in that rows are missing
df_tidy = df_tidy.dropna()
print("colonne ed entries after drop",df_tidy.shape)

colonne ed entries after drop (48842, 15)


In [5]:
#for age column --> Integer as String data smells
# Converti i valori non numerici in NaN
df_tidy['age'] = pd.to_numeric(df_tidy['age'], errors='coerce')

# Ora dovresti essere in grado di convertire in int
df_tidy['age'] = df_tidy['age'].astype(int)
print(df_tidy['age'].dtype)

int64


In [6]:
#for capital-gain column --> Integer as floating data smells
df_tidy['capital-gain'] = df_tidy['capital-gain'].astype(int)
print(df_tidy['capital-gain'].dtype)

int64


In [7]:
#for capital-loss column --> Integer as floating data smells
df_tidy['capital-loss'] = df_tidy['capital-loss'].astype(int)
print(df_tidy['capital-loss'].dtype)

int64


In [8]:
#for education-num column --> Integer as floating data smells
df_tidy['education-num'] = df_tidy['education-num'].astype(int)
print(df_tidy['education-num'].dtype)

int64


In [9]:
#for fnlwgt column --> Integer as floating data smells
df_tidy['fnlwgt'] = df_tidy['fnlwgt'].astype(int)
print(df_tidy['fnlwgt'].dtype)

int64


In [10]:
#for hours-per-week column --> Integer as floating data smells
df_tidy['hours-per-week'] = df_tidy['hours-per-week'].astype(int)
print(df_tidy['hours-per-week'].dtype)

int64


In [11]:
df_tidy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48842 entries, 0 to 48842
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      48842 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48842 non-null  object
 14  income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 6.0+ MB


In [12]:
int_columns = df_tidy.select_dtypes(include='int64')
print(int_columns)

       age  fnlwgt  education-num  capital-gain  capital-loss  hours-per-week
0       39   77516             13          2174             0              40
1       50   83311             13             0             0              13
2       38  215646              9             0             0              40
3       53  234721              7             0             0              40
4       28  338409             13             0             0              40
...    ...     ...            ...           ...           ...             ...
48838   39  215419             13             0             0              36
48839   64  321403              9             0             0              40
48840   38  374983             13             0             0              50
48841   44   83891             13          5455             0              40
48842   35  182148             13             0             0              60

[48842 rows x 6 columns]


In [13]:
#StandardScaler solo sulle colonne con extreme value
#scaler = StandardScaler()
#df_standardized = df_tidy.copy()
#df_standardized[['capital-gain', 'capital-loss', 'education-num', 'fnlwgt', 'hours-per-week']] = scaler.fit_transform(df_tidy[['capital-gain', 'capital-loss', 'education-num', 'fnlwgt', 'hours-per-week']])

In [14]:
scaler = MinMaxScaler()
df_minmax = df_tidy.copy()
df_minmax[['capital-gain', 'capital-loss', 'education-num', 'fnlwgt', 'hours-per-week','age']] = scaler.fit_transform(df_tidy[['capital-gain', 'capital-loss', 'education-num', 'fnlwgt', 'hours-per-week','age']])

In [15]:
df_minmax.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48842 entries, 0 to 48842
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             48842 non-null  float64
 1   workclass       48842 non-null  object 
 2   fnlwgt          48842 non-null  float64
 3   education       48842 non-null  object 
 4   education-num   48842 non-null  float64
 5   marital-status  48842 non-null  object 
 6   occupation      48842 non-null  object 
 7   relationship    48842 non-null  object 
 8   race            48842 non-null  object 
 9   sex             48842 non-null  object 
 10  capital-gain    48842 non-null  float64
 11  capital-loss    48842 non-null  float64
 12  hours-per-week  48842 non-null  float64
 13  native-country  48842 non-null  object 
 14  income          48842 non-null  object 
dtypes: float64(6), object(9)
memory usage: 6.0+ MB


In [16]:
df_minmax[['capital-gain', 'capital-loss', 'education-num', 'fnlwgt', 'hours-per-week','age']].head()

,capital-gain,capital-loss,education-num,fnlwgt,hours-per-week,age
0,0.02174,0.0,0.800000,0.044131,0.397959,0.301370
1,0.00000,0.0,0.800000,0.048052,0.122449,0.452055
2,0.00000,0.0,0.533333,0.137581,0.397959,0.287671
3,0.00000,0.0,0.400000,0.150486,0.397959,0.493151
4,0.00000,0.0,0.800000,0.220635,0.397959,0.150685


In [17]:
#df_minmax.info()
df_tidy = df_minmax.copy()
df_tidy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48842 entries, 0 to 48842
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             48842 non-null  float64
 1   workclass       48842 non-null  object 
 2   fnlwgt          48842 non-null  float64
 3   education       48842 non-null  object 
 4   education-num   48842 non-null  float64
 5   marital-status  48842 non-null  object 
 6   occupation      48842 non-null  object 
 7   relationship    48842 non-null  object 
 8   race            48842 non-null  object 
 9   sex             48842 non-null  object 
 10  capital-gain    48842 non-null  float64
 11  capital-loss    48842 non-null  float64
 12  hours-per-week  48842 non-null  float64
 13  native-country  48842 non-null  object 
 14  income          48842 non-null  object 
dtypes: float64(6), object(9)
memory usage: 6.0+ MB


In [18]:
category_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship','race','sex','native-country']
df_tidy = pd.get_dummies(df_tidy, columns=category_columns, drop_first=True)

In [19]:
pd.set_option('display.max_columns', None)
df_tidy['sex_Female'] = ~df_tidy['sex_Male'] # We add a new column that is the opposite
df_tidy['income'] = df_tidy['income'].map({'<=50K': 0, '>50K': 1}).astype(int)
df_tidy.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Male,native-country_Cambodia,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Holand-Netherlands,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,sex_Female
0,0.301370,0.044131,0.800000,0.02174,0.0,0.397959,0,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,0.452055,0.048052,0.800000,0.00000,0.0,0.122449,0,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2,0.287671,0.137581,0.533333,0.00000,0.0,0.397959,0,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [20]:
df_tidy.to_csv('adult/adultTidy2.csv', index=False)

In [21]:
X = df_tidy.drop(columns="income")
y = df_tidy["income"]

# Define four sets and apply the function
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, # 0.2 indicates a test set size of 20%
                                                    random_state=42)

In [22]:
dt_clf = DecisionTreeClassifier(random_state=42)

# The fit function will do the trick
dt_clf.fit(X_train, y_train)

# After the training phase, the model will be tested by predicting the values on the test set
dt_predictions = dt_clf.predict(X_test)

compute_performance_metrics(y_test,dt_predictions,"Decision Tree")

Decision Tree Accuracy: 0.8153342204933974
Decision Tree Recall: 0.6182590233545647
Decision Tree F1 Score: 0.6174724342663274


In [23]:
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='linear'))

# Addestra il classificatore sui dati di training
svm_classifier.fit(X_train, y_train)

# Fai delle predizioni sui dati di test
svm_pred = svm_classifier.predict(X_test)

compute_performance_metrics(y_test,svm_pred,"SVM")

SVM Accuracy: 0.8495240045040434
SVM Recall: 0.5605095541401274
SVM F1 Score: 0.6423357664233577


In [24]:
rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth = None, random_state=42)

rf_classifier.fit(X_train, y_train)

rf_predictions = rf_classifier.predict(X_test)

compute_performance_metrics(y_test,rf_predictions,"Random Forest")

Random Forest Accuracy: 0.8508547446002661
Random Forest Recall: 0.60552016985138
Random Forest F1 Score: 0.6618705035971223


In [25]:
privileged_attributeSex = "sex_Male"
unprivileged_attributesSex = ["sex_Female"]
privileged_attributeRace = "race_White"
unprivileged_attributesRace = ["race_Black", "race_Other", "race_Asian-Pac-Islander"]

In [26]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO IL DECISION TREE
dataset = X_test.copy(deep=True) # we create a copy of the test set
dataset['income'] = y_test  # and join the target feature with the others
predictions = dataset.copy(deep=True) # we do the same task
predictions['income'] = dt_predictions # but this time the target feature is made by the predictions of our model

In [27]:
#Attributo privilegiato la colonna sex_Male
#Attributo non privilegiato la colonna sex_Female
# Valore favorevole 1
# Valore non favorevole 0

fairness_metrics = compute_classification_metric(dataset,predictions,"income",1,0,privileged_attributeSex,unprivileged_attributesSex) #prima favorevole
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.193
Average Odds Difference (AOD): -0.101
Equal Opportunity Difference (EOD): -0.099


In [28]:
#Attributo privilegiato la colonna race_White
#Attributo non privilegiato tutte le colonne race != da race_White
#Valore di label favorevole 1 e label non favorevole 0

fairness_metrics = compute_classification_metric(dataset, predictions, "income", 1, 0, privileged_attributeRace, unprivileged_attributesRace)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.075
Average Odds Difference (AOD): -0.035
Equal Opportunity Difference (EOD): -0.046


In [29]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO L'SVM
dataset = X_test.copy(deep=True)
dataset['income'] = y_test
predictions = dataset.copy(deep=True) 
predictions['income'] = svm_pred

In [30]:
#Attributo privilegiato la colonna sex_Male
#Attributo non privilegiato la colonna sex_Female
# Valore favorevole 1
# Valore non favorevole 0

fairness_metrics = compute_classification_metric(dataset,predictions,"income",1,0,privileged_attributeSex,unprivileged_attributesSex) #prima favorevole
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.162
Average Odds Difference (AOD): -0.075
Equal Opportunity Difference (EOD): -0.083


In [31]:
#Attributo privilegiato la colonna race_White
#Attributo non privilegiato tutte le colonne race != da race_White
#Valore di label favorevole 1 e label non favorevole 0

fairness_metrics = compute_classification_metric(dataset, predictions, "income", 1, 0, privileged_attributeRace, unprivileged_attributesRace)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.062
Average Odds Difference (AOD): -0.018
Equal Opportunity Difference (EOD): -0.024


In [32]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO Il Random Forest
dataset = X_test.copy(deep=True)
dataset['income'] = y_test
predictions = dataset.copy(deep=True) 
predictions['income'] = rf_predictions

In [33]:
#Attributo privilegiato la colonna sex_Male
#Attributo non privilegiato la colonna sex_Female
# Valore favorevole 1
# Valore non favorevole 0

fairness_metrics = compute_classification_metric(dataset,predictions,"income",1,0,privileged_attributeSex,unprivileged_attributesSex) #prima favorevole
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.182
Average Odds Difference (AOD): -0.087
Equal Opportunity Difference (EOD): -0.091


In [34]:
#Attributo privilegiato la colonna race_White
#Attributo non privilegiato tutte le colonne race != da race_White
#Valore di label favorevole 1 e label non favorevole 0

fairness_metrics = compute_classification_metric(dataset, predictions, "income", 1, 0, privileged_attributeRace, unprivileged_attributesRace)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.085
Average Odds Difference (AOD): -0.033
Equal Opportunity Difference (EOD): -0.032
